In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer, SimpleImputer
from scipy.stats import loguniform, uniform
from pprint import pprint
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

## find performance upper bound
 
Making the assumption that the existing features has all the predictive power, we would like to examine whether dropping nan would improve model performance. In a way this tells us the performance upper bound of a model & whether getting the nan right is the key to success of prediction. 

In [4]:
PATH = '../data/raw'
df = pd.read_csv(os.path.join(PATH,'train.csv'))

In [5]:
def parse(df):
    return (
        df.assign(
            GroupSize = df['PassengerId'].str.split('_', expand=True)[1].astype(int),
            Deck = df['Cabin'].str.split('/',expand=True)[0],
            Side = df['Cabin'].str.split('/',expand=True)[2]
            )
        .drop(['Name','PassengerId','Cabin'],axis=1)
        )

## try several models aftering dropna. 

In [7]:
df = pd.read_csv(os.path.join(PATH,'train.csv'))
df = parse(df).dropna()

y = df.pop('Transported')
X = df

X_tr, X_dev, y_tr, y_dev = train_test_split(X,y,test_size=0.2, random_state=1123)

ohe = OneHotEncoder()
scaler = StandardScaler()


lr = LogisticRegression()
rf = RandomForestClassifier()
svc = SVC()
hgb = HistGradientBoostingClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
# cb = CatBoostClassifier(verbose=100)

ct = make_column_transformer(
    (ohe, ['HomePlanet','CryoSleep', 'Destination', 'VIP', 'GroupSize', 'Deck','Side']),
    (scaler, make_column_selector(dtype_include=np.number)),
    remainder='drop',
)

models = [lr, rf, svc, hgb,xgb,lgb]

for model in models:
    pipe = make_pipeline(
        ct, model
    )
    pipe.fit(X_tr,y_tr)
    y_pred = pipe.predict(X_dev)
    print(model.__class__.__name__)
    # print(classification_report(y_pred,y_dev))
    report = classification_report(y_pred,y_dev, output_dict=True) 
    display(pd.DataFrame(report).T)

LogisticRegression


,precision,recall,f1-score,support
False,0.768421,0.803459,0.785550,636.000000
True,0.818314,0.785216,0.801423,717.000000
accuracy,0.793792,0.793792,0.793792,0.793792
macro avg,0.793368,0.794338,0.793487,1353.000000
weighted avg,0.794861,0.793792,0.793962,1353.000000


RandomForestClassifier


,precision,recall,f1-score,support
False,0.818045,0.766197,0.791273,710.000000
True,0.758721,0.811820,0.784373,643.000000
accuracy,0.787879,0.787879,0.787879,0.787879
macro avg,0.788383,0.789008,0.787823,1353.000000
weighted avg,0.789852,0.787879,0.787994,1353.000000


SVC


...

c:\Users\bxxy002\miniconda3\envs\ds\Lib\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


XGBClassifier


,precision,recall,f1-score,support
0,0.786466,0.798473,0.792424,655.000000
1,0.808140,0.796562,0.802309,698.000000
accuracy,0.797487,0.797487,0.797487,0.797487
macro avg,0.797303,0.797517,0.797367,1353.000000
weighted avg,0.797647,0.797487,0.797524,1353.000000


[LightGBM] [Info] Number of positive: 2713, number of negative: 2698
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001633 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1412
[LightGBM] [Info] Number of data points in the train set: 5411, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501386 -> initscore=0.005544
[LightGBM] [Info] Start training from score 0.005544
LGBMClassifier


,precision,recall,f1-score,support
False,0.793985,0.804878,0.799394,656.000000
True,0.813953,0.803443,0.808664,697.000000
accuracy,0.804139,0.804139,0.804139,0.804139
macro avg,0.803969,0.804161,0.804029,1353.000000
weighted avg,0.804272,0.804139,0.804170,1353.000000


# Take away

 - dropna does some good, but improvement is not dramatic